## Setup

In [1]:
!pip install datasets -qq
!pip install transformers -qq
!pip install rouge_score evaluate nltk -qq

In [2]:
import torch
import numpy as np
import datasets
import nltk
import evaluate

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from collections import Counter
import matplotlib.pyplot as plt

## Load dataset

In [3]:
# load full xsum dataset
train_dataset, dev_dataset, test_dataset = datasets.load_dataset("glue", "qqp",split=['train', 'validation', 'test'])

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
filtered_train = train_dataset.filter(lambda example: example["label"]==1)
filtered_valid = dev_dataset.filter(lambda example: example["label"]==1)
# filtered_test = test_dataset.filter(lambda example: example["label"]==1)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [5]:
print("Number of paraphrased pairs in train_set:", len(filtered_train))
print("Number of paraphrased pairs in valid_set:", len(filtered_valid))
print()
print(f"Proportion of paraphrased pairs in train_set: {len(filtered_train)/len(train_dataset)*100:.2f}%")
print(f"Proportion of paraphrased pairs in valid_set: {len(filtered_valid)/len(dev_dataset)*100:.2f}%")


Number of paraphrased pairs in train_set: 134378
Number of paraphrased pairs in valid_set: 14885

Proportion of paraphrased pairs in train_set: 36.93%
Proportion of paraphrased pairs in valid_set: 36.82%


In [6]:
qqp = datasets.DatasetDict({
    'train': filtered_train,
    'validation': filtered_valid
})

In [7]:
qqp

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 134378
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 14885
    })
})

In [8]:
qqp["validation"][0]

{'question1': 'Is there a reason why we should travel alone?',
 'question2': 'What are some reasons to travel alone?',
 'label': 1,
 'idx': 2}

In [9]:
qqp["validation"].features

{'question1': Value(dtype='string', id=None),
 'question2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_duplicate', 'duplicate'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [10]:
# quora = datasets.load_dataset("quora")
# filter_quora = quora.filter(lambda example: example["is_duplicate"]==True)
# filter_quora

## Load pretrained BART and tokenizer

In [11]:
model_name = 'facebook/bart-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
# print(model.config)

In [13]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [14]:
print(f"BART base size: {np.round(model.num_parameters()/1e6, 1)} M parameters")

BART base size: 139.4 M parameters


## Tokenization

In [15]:
def batch_tokenize_preprocess(batch, tokenizer):
    source, target = batch["question1"], batch["question2"]
    source_tokenized = tokenizer(
        source, truncation=True
    )
    target_tokenized = tokenizer(
        target, truncation=True
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch

In [16]:
tokenized_datasets = qqp.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer
    ),
    batched=True,
    remove_columns=qqp['train'].column_names,
)

  0%|          | 0/135 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 134378
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14885
    })
})

In [18]:
tokenizer.model_max_length

1024

In [19]:
print(tokenized_datasets["train"][0])
print(qqp["train"][0])
print()
print(tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))
print(tokenizer.decode(tokenized_datasets["train"][0]["labels"]))

{'input_ids': [0, 6179, 109, 38, 797, 127, 46216, 8597, 116, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 6179, 109, 47, 797, 110, 21305, 6601, 116, 2]}
{'question1': 'How do I control my horny emotions?', 'question2': 'How do you control your horniness?', 'label': 1, 'idx': 1}

<s>How do I control my horny emotions?</s>
<s>How do you control your horniness?</s>


## Training

### Metrics: ROUGE

In [20]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [21]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

### Training arguments

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=2,  
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,
    gradient_accumulation_steps = 4,
    warmup_steps=500,
    weight_decay=0.1,
    evaluation_strategy='steps',
    eval_steps=500,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

### Train

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["sentence1"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

def get_response(input_text,num_return_sequences,num_beams):
    batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [ ]:
from huggingface_hub import login
login("...")

In [ ]:
model_repo = "harouzie/bart-paraphraser"
model.push_to_hub(model_repo)
tokenizer.push_to_hub(model_repo)